

# Developing Fair Classifiers: A case study using COMPAS data

## Introduction

##### There are many different ways to measure fairness in AI. As such, there are many ways to adjust your AI models so that they better fit these fairness metrics. Here, you will examine this firsthand with the COMPAS dataset, which is a dataset that has been widely used to predict inmates' likelihood of recidivism (reoffense). First, you will run a simple classifier on this dataset and examine the results. Then, you will be asked how this classifier could be made more fair and, for extra credit, you can try implementing these changes yourself.


## Dataset exploration

##### First, you will run the cells and answer the questions below in order to explore the contents of the dataset

### Imports

In [ ]:
#uncomment these lines if you need to install either of these two packages

#!pip install plotly
#!pip install aif360[LFR]

In [ ]:
import urllib
import urllib.request
import os,sys
import numpy as np
import pandas as pd
from functools import reduce
pd.set_option('display.max_columns',None)
from pprint import pprint
import matplotlib.pyplot as plt
import sklearn.metrics as performance
from sklearn.preprocessing import scale
from random import seed, shuffle
from IPython.display import Markdown, display

%matplotlib inline

try:
    import plotly.express as px
    import plotly.figure_factory as ff
    import plotly.io as pio
    import plotly.graph_objects as go
    pio.renderers.default = "colab" #change to notebook_connected if in a Jupyter notebook
except ImportError as e:
#     !conda install --yes --prefix {sys.prefix} -c plotly plotly-orca==1.2.1 psutil requests
    print("plotly is not installed !! please install the package to be able to render plotly visualizations")


try:
    from aif360.metrics import BinaryLabelDatasetMetric
    from aif360.metrics import ClassificationMetric
    from aif360.metrics.utils import compute_boolean_conditioning_vector
    from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing

    from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
                    import load_preproc_data_compas
    
except ImportError as e:
    print('please install aif360 package to proceed with the fairness section')
    


### Setting up visualization properties

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
fig=plt.figure(figsize=(10,6))
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title



### UTILS

In [ ]:
def hist(column_name, x):
    '''
    helper function to plot historgram for a dataframe 
    '''
    return px.histogram(column_name, x = x,
                       title = x,
                       opacity = 0.8,
                       color_discrete_sequence = ['indianred'])
def plot_roc(fpr,tpr,thresholds=None,label=None):
    '''
    plot roc curve for multiple instances.
    '''
    
    colors = ['red','blue','aqua', 'darkorange', 'cornflowerblue', 'burlywood', 'lightsalmon', 'olive']

    plt.figure(figsize=(12,8))
    lw = 2
    for i in range(len(fpr)):
        plt.plot(fpr[i], tpr[i], color=colors[i],label=label[i],
             lw=lw)
        
     
    plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
      
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves of attributes ')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.legend(loc="lower right")
    plt.show()



### Download data file and dependency files

In [ ]:
SEED = 1234
seed(SEED)
np.random.seed(SEED)

def check_data_file(fname):
    files = os.listdir(".") # get the current directory listing
    print("Looking for file %s in the current directory..." % (fname))

    if fname not in files:
        print(" %s not found! Downloading from GitHub..." % fname)
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        response = urllib.request.urlopen(addr)
        data = response.read()
        fileOut = open(fname, "wb")
        fileOut.write(data)
        fileOut.close()
        print("%s download and saved locally.." % fname)
    else:
        print("File found in current directory..")
    
COMPAS_INPUT_FILE = "compas-scores-two-years.csv"
check_data_file(COMPAS_INPUT_FILE)  

### Load and clean data

In [ ]:
dataset = pd.read_csv(COMPAS_INPUT_FILE)

dataset = dataset.dropna(subset=["days_b_screening_arrest"]) # dropping missing vals

dataset = dataset[ (dataset.days_b_screening_arrest <= 30) &
(dataset.days_b_screening_arrest >= -30) &
(dataset.is_recid != -1) & (dataset.c_charge_degree != 'O') & (dataset.score_text != 'N/A') ]

dataset.reset_index(inplace=True, drop=True) # renumber the rows from 0 again

In [ ]:
dataset.head(10)

### Basic analysis of demographics
A series of plots is given below which look at the distribution of the data, the decile scores (the scores in the COMPAS dataset that measure likelihood of reoffense), and the risk category, which is based on the decile score and is determined in the following manner:

 1 – 4: scale score is low relative to other offenders in norm group.

 5 – 7: scale score is medium relative to other offenders in norm group.

 8 – 10: scale score is high relative to other offenders in norm group.


After looking at the plots, you will be asked to describe the imbalances of the dataset

In [ ]:
def get_basic_stats(column_name,check_values=None,
                    check_NaN=False,group_count=True,plot_hist = False):
    '''
    basic stats on a given column of the dataset
    @param column_name: column name to compute stats on
    '''
    
    result = {}
    
    if column_name not in dataset.columns:
        raise ValueError("column_name must be set to a value from the the available values")
        
    #values in the column
    column_values = np.unique(dataset[column_name].values)
    
    if check_values:
        print('-',column_name,"in dataset:")
        [print(value) for value in column_values]

    if check_NaN:
        print("-NaN  present in the column?",dataset[column_name].isnull().any())
    
    if group_count:
        values_count = dataset[column_name].value_counts()   
        result.update(values_count=values_count)
        
    #histogram of values
    if plot_hist:
        fig = hist(dataset[column_name], column_name) 
        fig.show()        
    
    return result

In [ ]:
result = get_basic_stats("race",plot_hist=True)

In [ ]:
result = get_basic_stats("age", plot_hist=True)

In [ ]:
sex_race_df = pd.crosstab(dataset.race, dataset.sex)
sex_race_df = sex_race_df.stack().reset_index().rename(columns={0:"value"})

In [ ]:
race = dataset.race.unique()
males = sex_race_df.value[sex_race_df.sex == 'Male']
females = sex_race_df.value[sex_race_df.sex == 'Female']

fig = go.Figure(data=[
    go.Bar(name='Male', x=race, y= males),
    go.Bar(name='Female', x=race, y=females)
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
decile_score_per_race = pd.crosstab(dataset.decile_score, dataset.race)
decile_score_per_race = decile_score_per_race.stack().reset_index().rename(columns = {0:'values'})

In [ ]:
dec_score = dataset.decile_score.unique()
afrac_amer = decile_score_per_race.values[decile_score_per_race.race == "African-American"][:,2]
hispanic = decile_score_per_race.values[decile_score_per_race.race == "Hispanic"][:,2]
asian = decile_score_per_race.values[decile_score_per_race.race == "Asian"][:,2]
causcasian = decile_score_per_race.values[decile_score_per_race.race == "Caucasian"][:,2]
native = decile_score_per_race.values[decile_score_per_race.race == "Native American"][:,2]
other = decile_score_per_race.values[decile_score_per_race.race == "Other"][:,2]

fig = go.Figure(data = [
    go.Bar(name = 'African-American', x = dec_score, y = afrac_amer),
    go.Bar(name = 'Hispanic', x = dec_score, y = hispanic),
    go.Bar(name = 'Asian', x = dec_score, y = asian),
    go.Bar(name = 'Causcasian', x = dec_score, y = causcasian),
    go.Bar(name = 'Native-American', x = dec_score, y = native),
    go.Bar(name = 'Other', x = dec_score, y = other)
])
fig.update_layout(barmode='stack')
fig.show()

The *low*, *medium* and *high* values below are categories for risk based on the **decile_score** computed. This will be discussed in more detail in the upcoming sections.

In [ ]:
score_text_per_race = pd.crosstab(dataset.score_text, dataset.race)
score_text_per_race = score_text_per_race.stack().reset_index().rename(columns = {0:'values'})

In [ ]:
dec_score = dataset.score_text.unique()
afrac_amer = score_text_per_race.values[score_text_per_race.race == "African-American"][:,2]
hispanic = score_text_per_race.values[score_text_per_race.race == "Hispanic"][:,2]
asian = score_text_per_race.values[score_text_per_race.race == "Asian"][:,2]
causcasian = score_text_per_race.values[score_text_per_race.race == "Caucasian"][:,2]
native = score_text_per_race.values[score_text_per_race.race == "Native American"][:,2]
other = score_text_per_race.values[score_text_per_race.race == "Other"][:,2]

fig = go.Figure(data = [
    go.Bar(name = 'African-American', x = dec_score, y = afrac_amer),
    go.Bar(name = 'Hispanic', x = dec_score, y = hispanic),
    go.Bar(name = 'Asian', x = dec_score, y = asian),
    go.Bar(name = 'Causcasian', x = dec_score, y = causcasian),
    go.Bar(name = 'Native-American', x = dec_score, y = native),
    go.Bar(name = 'Other', x = dec_score, y = other)
])
fig.update_layout(barmode='stack')
fig.show()

### Decile score vs Race


In [ ]:
def scores_stats(category, value):
    '''
    @param category: vategory to perform stats on
    @param value: entry in the specefied category
    '''
    scores = dataset[dataset[category] == value]
    scores = scores[['decile_score','score_text']]
    histogram,_ = np.histogram(scores['decile_score'])
    fig = px.histogram(scores['decile_score'], 'decile_score',
                       title = 'Decile score of '+ value + 's',
                       opacity = 0.8,
                       color_discrete_sequence = ['indianred'])
    fig.add_shape(dict(
        type="line",
        x0 = scores['decile_score'].mean(),
        y0 = 0,
        x1 = scores['decile_score'].mean(),
        y1 = max(histogram) + 20,
        line=dict(
            color = "LightSeaGreen",
            width = 4,
            dash = "dashdot")
    ))
    fig.show()
    print('mean decile score: %.2f'%scores['decile_score'].mean())


In [ ]:
scores_stats('race','African-American')

In [ ]:
scores_stats('race','Caucasian')

### Decile score vs Gender


In [ ]:
scores_stats('sex','Male')

In [ ]:
scores_stats('sex','Female')

### **Question 1: Takeaways from the data**
Now that you have looked at the way this data is distributed, what do you notice? What imbalances (if any) do you see and how might they affect the results of a model that uses this dataset? Answer these questions in the cell below:



### Performance analysis of COMPAS


Now, we are going to measure COMPAS's overall performance and then break it down by gender and race. We do this by comparing the COMPAS scores with the ground truth data regarding whether or not a person was re-arrested for an offense within two years (what COMPAS is trying to predict)

#### Number of people who were re-arrested


In [ ]:
#people whom got re-arrested in two years span
print('Number of people in the dataset whom got re-arrested in the span of two years: ',
      dataset['two_year_recid'].sum())

#### Number of people who were rearrested, split by race and gender

In [ ]:
# recedivism by race
cau_sum = dataset[(dataset['race'] == 'Caucasian') & (dataset['two_year_recid']==1)]['two_year_recid'].sum()
print('Number of Caucasians in the dataset whom got re-arrested in the span of two years and \
also predicted to recid: {0}/{1}'.format(cau_sum,dataset['race'].value_counts()['Caucasian']))


In [ ]:
am_sum = dataset[(dataset['race'] == 'African-American') & (dataset['two_year_recid']==1)]['two_year_recid'].sum()
print('Number of African Americans in the dataset whom got re-arrested in the span of two years and \
also predicted to recid: {0}/{1}'.format(am_sum,dataset['race'].value_counts()['African-American']))


In [ ]:
#recedivism by gender
male_sum = dataset[(dataset['sex'] == 'Male') & (dataset['two_year_recid']==1)]['two_year_recid'].sum()
print('Number of males in the dataset whom got re-arrested in the span of two years and \
also predicted to recid: {0}/{1}'.format(male_sum,dataset['sex'].value_counts()['Male']))


In [ ]:
female_sum = dataset[(dataset['sex'] == 'Female') & (dataset['two_year_recid']==1)]['two_year_recid'].sum()
print('Number of females in the dataset whom got re-arrested in the span of two years and \
also predicted to recid:{0}/{1}'.format(female_sum,dataset['sex'].value_counts()['Female']))


In [ ]:
two_recide_sex_race_df = pd.crosstab(dataset.race[dataset.two_year_recid == 1], dataset.sex[dataset.two_year_recid == 1])
two_recide_sex_race_df = two_recide_sex_race_df.stack().reset_index().rename(columns={0:"value"})

In [ ]:
race = dataset.race.unique()
race = np.sort(race)
males = two_recide_sex_race_df.value[two_recide_sex_race_df.sex == 'Male']
females = two_recide_sex_race_df.value[two_recide_sex_race_df.sex == 'Female']

fig = go.Figure(data=[
    go.Bar(name='Male', x=race, y= males),
    go.Bar(name='Female', x=race, y=females)
], layout_title_text = "Male/Female Recidivism by Race")
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
# Plot confusion matrix without passing the classifier
from sklearn.metrics import confusion_matrix
from itertools import cycle, product
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = 'd'
    thresh = cm.max() / 2
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
def plot_conf_matrix(y_test, y_pred, class_names):
    print("plotting the Confusion Matrix")
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    print('CCR = {}'.format(np.trace(cnf_matrix) / len(y_test)))
    print('Precision = {}'.format(precision_macro_average(cnf_matrix)))
    print('Recall = {}'.format(recall_macro_average(cnf_matrix)))
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion Matrix')
    plt.show()
    
def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

In [ ]:
#mapping of low medium high to decile score
print('decile score ranges in score text')
scores = dataset[['decile_score','score_text']]
low_range = scores[scores['score_text'] == 'Low']['decile_score']
print('low:',low_range.min(),low_range.max())
medium_range = scores[scores['score_text'] == 'Medium']['decile_score']
print('medium:',medium_range.min(),medium_range.max())
High_range = scores[scores['score_text'] == 'High']['decile_score']
print('high',High_range.min(),High_range.max())

To be able to compare compas scores in range [1,10], to the ground truth binary 0/1, the COMPAS scores are grouped into two groups to transfer them into binary labels
- *recidivate* means *high* = label *1*
- *will not recidivate* means *low/medium* = label *0*

In [ ]:
# we split scores to new classes, high to 1 and medium/ low to zero
new_scores = dataset.copy()
new_scores['decile_score'].replace(to_replace=range(1,8),value=0,inplace=True)
new_scores['decile_score'].replace(to_replace=range(8,11),value=1,inplace=True)


#### Overall Accuracy of COMPAS Scores

In [ ]:
overal_accuracy = performance.accuracy_score(new_scores['two_year_recid'],
                                             new_scores['decile_score'],normalize=True)

print('COMPAS overall performance against the true labels: {0:3f}%'.format(overal_accuracy *100))

## Part 2.2 - Standard classifiers

In this section, we are going to train different classifiers on the "two_years_recid" ground truth data and compare their performance, then plot the roc curves to investigate the bias in the classifiers towards different attributes. 

- Preparing for the next part, we start using IBM aif360 package, a collection of fairness algorithms and performance metrics  

- We uses the following set of features out of the 137 questions collected by northepoint, we notice that we are including the charge degree(Felony/murder), the priors count(number of crimes commited before) nad age category.

In [ ]:
dataset_conditioned = load_preproc_data_compas()
print("set of features to be used out of the dataset")
pprint(dataset_conditioned.feature_names)    

- As a normalization step, all of the data values are turned into binary values, in order to equalize the effect of features with respect to the classifier.

- The race has more than one value, but since we are only interested in the sensitive attributes (bias between African American and Caucasians, we drop the other races from the dataset, this does not in turn introduce a problem, since the size of the dataset drops from 6127 entry to 5278 entry.

In [ ]:
dataset_as_df,_ = dataset_conditioned.convert_to_dataframe()
print("Example samples from the dataset")
dataset_as_df.head(5)

### Train-validate-test split

- We divide our dataset into 3 folds to train/test different classifers

In [ ]:
dataset_conditioned_train, dataset_conditioned_validate_test = dataset_conditioned.split([0.7], shuffle=True)
# dataset_conditioned_validate, dataset_conditioned_test = dataset_conditioned_validate_test.split([0.5], shuffle=True)



In [ ]:
print('size of traning and testing samples')
dataset_conditioned_train.convert_to_dataframe()[0].shape,dataset_conditioned_validate_test.convert_to_dataframe()[0].shape

### Splitting based on gender 

In [ ]:
# get male samples from the dataset
male_samples = dataset_as_df[dataset_as_df['sex'] == 1.0]
male_samples_label = male_samples.filter(['two_year_recid'])
male_samples.drop(columns=['two_year_recid'],inplace=True)

# get femal samples from the dataset
female_samples = dataset_as_df[dataset_as_df['sex'] == 0.0]
female_samples_label = female_samples.filter(['two_year_recid'])
female_samples.drop(columns=['two_year_recid'],inplace=True)
print('number of female samples {0},number of male samples {1}'.format(female_samples.shape,
                                                                                      male_samples.shape))

### Splitting based on race - African American and Caucasian

In [ ]:

#get caucasians samples from the dataset
caucasian_samples = dataset_as_df[dataset_as_df['race'] == 1.0]
caucasian_samples_label = caucasian_samples.filter(['two_year_recid'])
caucasian_samples.drop(columns=['two_year_recid'],inplace=True)

#get african american samples from the dataset
african_american_samples = dataset_as_df[dataset_as_df['race'] == 0.0]
african_american_samples_label = african_american_samples.filter(['two_year_recid'])
african_american_samples.drop(columns=['two_year_recid'],inplace=True)
print('number of african american samples {0},number of caucasians samples {1}'.format(african_american_samples.shape,
                                                                                      caucasian_samples.shape))

### Accuracy of COMPAS Scores

Before delving into the performance of different types of classifiers, we plot the ROC curves of compas scores with respect to race and gender in order to inspect the true positive rate and false positive rates.

#### ROC Curve - African American vs. Caucasian


In [ ]:
fpr_AM,tpr_AM,thresholds_AM = performance.roc_curve(new_scores_african_american['two_year_recid'],
                                           new_scores_african_american['decile_score'],pos_label=1)

fpr_C,tpr_C,thresholds_C = performance.roc_curve(new_scores_Caucasian['two_year_recid'],
                                           new_scores_Caucasian['decile_score'],pos_label=1)

lw = 2
fig=plt.figure(figsize=(10,6))
plt.plot(fpr_AM, tpr_AM, color='darkorange',label='ROC curve of African americans',
         lw=lw)
plt.plot(fpr_C, tpr_C, color='r',
         lw=lw,label='ROC curve of Caucasians')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

#### ROC curve for gender

In [ ]:
fpr_female,tpr_female,thresholds_female = performance.roc_curve(new_scores_female['two_year_recid'],
                                           new_scores_female['decile_score'],pos_label=1)

fpr_male,tpr_male,thresholds_male = performance.roc_curve(new_scores_male['two_year_recid'],
                                           new_scores_male['decile_score'],pos_label=1)

lw = 2
fig=plt.figure(figsize=(10,6))
plt.plot(fpr_female, tpr_female, color='darkorange',label='ROC curve of females',
         lw=lw)
plt.plot(fpr_male, tpr_male, color='r',
         lw=lw,label='ROC curve of males')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

###**Question 2: What biases do you see in the COMPAS data based on these ROC curves?**
Answer this question in the cell below

Next, we examine the results of running three simple classifiers on the COMPAS data

### SVM


In [ ]:
from sklearn.svm import SVC
svm_classifier = SVC(gamma=0.001, C=100.)
svm_classifier.fit(dataset_conditioned_train.features, dataset_conditioned_train.labels.ravel())

pred = svm_classifier.predict(dataset_conditioned_validate_test.features)

print('Accuracy of the SVM on testing set {0:.2f}% '
      .format(performance.accuracy_score(dataset_conditioned_validate_test.labels, pred, normalize=True)*100))


#### ROC curve for race

In [ ]:
# ROC curve for black vs caucasian
fpr_svm_race,tpr_svm_race,label_svm_race=[],[],[]


african_american_prediction = svm_classifier.predict(african_american_samples)

fpr_AM,tpr_AM,thresholds_AM = performance.roc_curve(african_american_samples_label.to_numpy().ravel(),
                                           african_american_prediction,pos_label=1)

fpr_svm_race.append(fpr_AM)
tpr_svm_race.append(tpr_AM)
label_svm_race.append('ROC of african american')

caucasian_prediction = svm_classifier.predict(caucasian_samples)
fpr_C,tpr_C,thresholds_C = performance.roc_curve(caucasian_samples_label.to_numpy().ravel(),
                                           caucasian_prediction,pos_label=1)
fpr_svm_race.append(fpr_C)
tpr_svm_race.append(tpr_C)
label_svm_race.append('ROC of Caucasians')

plot_roc(fpr_svm_race,tpr_svm_race,label=label_svm_race)


#### ROC curve for gender

In [ ]:
# ROC curve for male vs female
fpr_svm_gender,tpr_svm_gender,label_svm_gender=[],[],[]


female_prediction = svm_classifier.predict(female_samples)

fpr_female,tpr_female,thresholds_female = performance.roc_curve(female_samples_label.to_numpy().ravel(),
                                           female_prediction,pos_label=1)

fpr_svm_gender.append(fpr_female)
tpr_svm_gender.append(tpr_female)
label_svm_gender.append('ROC of females')

male_prediction = svm_classifier.predict(male_samples)
fpr_male,tpr_male,thresholds_male = performance.roc_curve(male_samples_label.to_numpy().ravel(),
                                           male_prediction,pos_label=1)
fpr_svm_gender.append(fpr_male)
tpr_svm_gender.append(tpr_male)
label_svm_gender.append('ROC of male')

plot_roc(fpr_svm_gender,tpr_svm_gender,label=label_svm_gender)


### MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

hidden_layer_grid = [(x,x) for x in range(10,16,2)]

mlp_classifier = MLPClassifier()
mlp_clf_opt = GridSearchCV(mlp_classifier,param_grid={'hidden_layer_sizes':hidden_layer_grid,
                                                     'solver':['adam'],
                                                      'learning_rate_init': [0.001],
                                                      'learning_rate':['adaptive'],
                                                      'momentum':[0.9], 
                                                      'batch_size': [16]})

mlp_clf_opt.fit(dataset_conditioned_train.features, dataset_conditioned_train.labels.ravel())
pred = mlp_clf_opt.predict(dataset_conditioned_validate_test.features)

model_accuracy = performance.accuracy_score(dataset_conditioned_validate_test.labels, pred, normalize=True)*100
print(' Accuracy of MLP on training set: {0:.2f}% '.format(model_accuracy))


#### ROC curve for race

In [ ]:
# ROC curve for black vs caucasian

fpr_mlp_race,tpr_mlp_race,label_mlp_race=[],[],[]


african_american_prediction = mlp_clf_opt.predict(african_american_samples)

fpr_AM,tpr_AM,thresholds_AM = performance.roc_curve(african_american_samples_label.to_numpy().ravel(),
                                           african_american_prediction,pos_label=1)

fpr_mlp_race.append(fpr_AM)
tpr_mlp_race.append(tpr_AM)
label_mlp_race.append('ROC of african american')

caucasian_prediction = mlp_clf_opt.predict(caucasian_samples)
fpr_C,tpr_C,thresholds_C = performance.roc_curve(caucasian_samples_label.to_numpy().ravel(),
                                           caucasian_prediction,pos_label=1)
fpr_mlp_race.append(fpr_C)
tpr_mlp_race.append(tpr_C)
label_mlp_race.append('ROC of Caucasians')

plot_roc(fpr_mlp_race,tpr_mlp_race,label=label_mlp_race)


#### ROC curve for gender

In [ ]:
# ROC curve for male vs female
fpr_mlp_gender,tpr_mlp_gender,label_mlp_gender=[],[],[]


female_prediction = mlp_clf_opt.predict(female_samples)

fpr_female,tpr_female,thresholds_female = performance.roc_curve(female_samples_label.to_numpy().ravel(),
                                           female_prediction,pos_label=1)

fpr_mlp_gender.append(fpr_female)
tpr_mlp_gender.append(tpr_female)
label_mlp_gender.append('ROC of females')

male_prediction = mlp_clf_opt.predict(male_samples)
fpr_male,tpr_male,thresholds_male = performance.roc_curve(male_samples_label.to_numpy().ravel(),
                                           male_prediction,pos_label=1)
fpr_mlp_gender.append(fpr_male)
tpr_mlp_gender.append(tpr_male)
label_mlp_gender.append('ROC of male')

plot_roc(fpr_mlp_gender,tpr_mlp_gender,label=label_mlp_gender)


### Decision trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree_classifier = DecisionTreeClassifier()

tree_clf_opt = GridSearchCV(tree_classifier,param_grid={'max_depth':[4,6,8,10,12]})
tree_clf_opt.fit(dataset_conditioned_train.features, dataset_conditioned_train.labels.ravel())

pred = tree_clf_opt.predict(dataset_conditioned_validate_test.features)

print('Accuracy of the Decision tree on testing set {0:.2f}% '
      .format(performance.accuracy_score(dataset_conditioned_validate_test.labels, pred, normalize=True)*100))


#### ROC curve for race

In [ ]:
# ROC curve for black vs caucasian

fpr_Dec_tree_race,tpr_Dec_tree_race,label_Dec_tree_race=[],[],[]


african_american_prediction = tree_clf_opt.predict(african_american_samples)

fpr_AM,tpr_AM,thresholds_AM = performance.roc_curve(african_american_samples_label.to_numpy().ravel(),
                                           african_american_prediction,pos_label=1)

fpr_Dec_tree_race.append(fpr_AM)
tpr_Dec_tree_race.append(tpr_AM)
label_Dec_tree_race.append('ROC of african american')

caucasian_prediction = tree_clf_opt.predict(caucasian_samples)
fpr_C,tpr_C,thresholds_C = performance.roc_curve(caucasian_samples_label.to_numpy().ravel(),
                                           caucasian_prediction,pos_label=1)
fpr_Dec_tree_race.append(fpr_C)
tpr_Dec_tree_race.append(tpr_C)
label_Dec_tree_race.append('ROC of Caucasians')

plot_roc(fpr_Dec_tree_race,tpr_Dec_tree_race,label=label_Dec_tree_race)


#### ROC curve for gender


In [ ]:
# ROC curve for male vs female
fpr_Dec_tree_gender,tpr_Dec_tree_gender,label_Dec_tree_gender=[],[],[]


female_prediction = tree_clf_opt.predict(female_samples)

fpr_female,tpr_female,thresholds_female = performance.roc_curve(female_samples_label.to_numpy().ravel(),
                                           female_prediction,pos_label=1)

fpr_Dec_tree_gender.append(fpr_female)
tpr_Dec_tree_gender.append(tpr_female)
label_Dec_tree_gender.append('ROC of females')

male_prediction = tree_clf_opt.predict(male_samples)
fpr_male,tpr_male,thresholds_male = performance.roc_curve(male_samples_label.to_numpy().ravel(),
                                           male_prediction,pos_label=1)
fpr_Dec_tree_gender.append(fpr_male)
tpr_Dec_tree_gender.append(tpr_male)
label_Dec_tree_gender.append('ROC of male')

plot_roc(fpr_Dec_tree_gender,tpr_Dec_tree_gender,label=label_Dec_tree_gender)


### **Question 3: What biases did the classifiers show?**
After training and testing these simple classifiers, what did you notice about the results above? How did they compare to the ROC curves from the COMPAS data? Answer these questions in the cell below

## Building fairer classifiers

### **Question 4: How can these classifiers be improved?**
What modifications can be made on the above classifiers that would result in fairer outcomes? Write two different potential modifications that could be made in the cell below

###**Extra Credit - Implement a Fairer classifier**
Using your suggested modifications above, modify one of the simple classifiers in order to produce a fairer model. Implement this model and plot the resulting ROC curves below